## A Brief Introduction to Multilayer Perceptions
A Perceptron is a single neuron model that was a precursor to larger neural networks. A perceptron is able to receive an single dimensional array of inputs and produce an single dimensional array of outputs that can be used for a variety of ML tasks, the most fundamental of which is classification. 

The building block for neural networks are artificial neurons. These are simple computational units that have weighted input signals and produce an output signal using an activation function. 

The weights attached to input signals can be likened to the coefficients of a regression equation. These weights represent the level of importance for each input. Consequently, changes in some inputs will have a larger effect on the output of the perceptron than others. Furthermore, each neuron will have a bias, a scalar weight that adjusts the final output of the neuron by a set amount, positive or negative.

Weights are often initialized to small random values although more complex initialization schemes can be used. Larger weights tend to increase the complexity and fragility of the model, so it is desirable to keep weights small and regularization techniques can be used to decrease the risks of overfitting. 

### Activation

The weighted inputs are summed and passed through an **activation function**, sometimes called a **transfer function**. It is called an activation function because it governs the threshold at which the neuron is "activated" and the strength of the output signal. The output of the function is limited in various ways, from application of the sigmoid function to bind values between 0 and 1, to hyperbolic tangent functions (Tanh) that binds values between -1 and 1, to ReLU (rectifier activation function) that binds values between 0 and another positive number. ReLU has been found to provide better results, generally, and this may be due to its mimicry of all-or-nothing type activation systems found in biological neurology. 

### Network of Neurons

Neurons are arranged into networks of neurons. A row of neurons is called a **layer** and one network can have multiple layers. The architecture of the neurons in the network is often called the **network topology**. 

### Input of Visible Layers

The bottom layer that takes input from a dataset is called the visible layer, because it is the exposed part of the network. Fundamentally, it is simply the series of initial inputs, where each input is the value from each column in the dataset for a given observation. 

### Hidden Layers
Layers after the input layer are called **hidden layers** because they are not directly exposed to the input. Each input is effectively transformed through a weight and bias. 

### Output Layer

The final hidden layer is called the output layer and is responsible for producing a value or vector of values that correspond to the format required for the problem. Here are some examples:
- A regression problem may have a single output neuron and the neuron may have no activation function.
- A binary classification problem may have a single output neuron and use a sigmoid activation function to output a value between 0 and 1 to represent the probability of predicting a value for the primary class. This can be turned into a crisp class value by using a threshold of 0.5 and snap values less than the threshold to 0 otherwise to 1. 
- A multiclass classification problem may have multiple neurons in the output layer, one for each class (e.g. three neurons for the three classes in the famous iris flowers classification problem). In this case, a softmax activation function may be used to output a probability of the network predicting each of the class values. Selecting the output with the highest probability can be used to produce a crisp class classification value. 

## Training Networks
Once configured, the neural network must be trained on the dataset.

### Data Preparation
- Data must be numerical
    - Categorical data can be converted via *one-hot encoding*.
    - Text data will need to be converted via TF-IDF or some other NLP encoding technique.
    - This same one hot encoding can be used on the output variable in classification problems with more than one class.
- Data must be similarly scaled
    - Min-max scaling (or any other scaling transformation) can be applied to the dataset to ensure consistency between inputs
    
### Stochastic Gradient Descent
The classical training algorithm for neural networks is stochastic gradient descent. A single observation is exposed to the network, and the output is generated (called a **forward pass**). The output is compared to the expected output and an error is calculated. The error is propagated back through the network, one layer at a time, and the weights are updated according to the amount they contributed ot the error (known as **backpropagation**). One round of updating the network for the entire training dataset is called an **epoch**. A network may be trained for any number of epochs. 

### Weight Updates
The weights can be updated from the errors calculated after each forward pass for each observation. This is called **online learning**. Alternatively, the errors can be aggregated across all results of all observations in the training set. This is called **batch learning** and is often more stable. 

Because datasets are so large, the size of the batch will be reduced before an update is completed. The amount that the weights are updated is controlled by a configuration parameter called the learning rate. This is also called the **step size** and controls the momentum at which a local minima is approached in order to reduce the risk of a convergence failure (where a local minima cannot be identified because each update overshoots a true local minima of the aggregated errors). 
- **Momentum** is a term that incorporates the properties from the previous weight update to allow the weights to continue to change in the same direction even when there is less error being calculated.
- **Learning Rate Decay** is used to decrease the learning weight over epochs to allow the network to make large changes to the weights at the beginning and smaller fine tuning changes later in the training schedule.

### Prediction
Predictions are made by providing the input to the network and performing a forward-pass allowing it to generate an ouput that you can use as a prediction. Evaluating performance on a separate out-of-sample dataset (ideally randomly split from the original full dataset available) can alert to risks of overfitting.

The network topology and the final set of weights is all that needs to persist from the research environment into the production environment to make novel predictions. 

## Developing a Simple Neural Network from the Pima Indians Onset of Diabetes Dataset

This is a standard machine learning dataset available from the UCI Machine Learning repository. It describes patient medical record data for Pima Indians and whether they had an onset of diabetes within five years. It is a binary classification problem (onset of diabetes as 1 or not as 0). Below are the attributes for the dataset:
1. Number of times pregnant.
2. Plasma glucose concentration at 2 hours in an oral glucose tolerance test.
3. Diastolic blood pressure (mm Hg).
4. Triceps skin fold thickness (mm).
5. 2-Hour serum insulin (mu U/ml).
6. Body mass index.
7. Diabetes pedigree function.
8. Age (years).
9. Class, onset of diabetes within five years.

## Loading Data

In [1]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

In [2]:
# loading the dataset
dataset = loadtxt('pima-indians-diabetes.csv', delimiter=',')

dataset

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]])

In [3]:
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

## Defining the Model

Models in Keras are a sequence of layers. We create a `Sequential` model and add layers one at a time until we are happy with our network topology. The first thing to get right is to ensure the input layer has the right number of inputs. This can be specified when creating the first layer with the `input_dim` argument and setting it to `8` for the 8 input variables. 

How do we know the number of layers to use and their types? Trial and error experimentation. Generally, you need a network large enough to capture the structure of the problem. In this example, we will use a fully-connected network structure with three layers.

Fully connected layers are defined using the `Dense` class. We can specify the number f neurons in the layer as the first argument and specify the activation function using the `activation` argument. We will use the rectifier (`relu`) activation function on the first two layers and the `sigmoid` activation function in the output layer. It used to be the case that sigmoid and `tanh` activation functions were preferred for all layers. These days, better performance is seen using the rectifier activation function. For the output layer, we can snap the classification by setting a threshold, in this case 0.5.

The first hidden layer will have 12 neurons and expect 8 input variables (e.g. `input_dim=8`). The second hidden layer will have 8 neurons and finally the output layer will have 1 neuron to predict the class.

In [4]:
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

## Compiling the Model

The backend will automatically choose the best way to represent the network for training and making predictions to run on my hardware. When compiling, we must specify some additional properties required when training the network:
- The loss function to used to evaluate a set of weights
- The optimizer used to serach through different weights 
- Any optional metrics we would like to collect and report

In this case, we will use logarithmic loss, which for a binary classification problem is defined in Keras as `binary_crossentropy`. We will use the gradient descent algorithm `adam` as it is an efficient default. For metrics, we will capture and report the model accuracy.

In [5]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Fit the Model

We have defined our model and compiled it for efficient computation. Now it is time to execute the model on some data. We can train on our loaded dat by calling the `fit()` function on the model.

The training process will run for a fixed number of iterations through the dataset called epochs, that we must specify with the `epochs` argument. We can also set the number of instances that are evaluated before a weight update in the network is performed called the batch sixe and set using the `batch_size` argument. For this problem we will run a small number of epochs (150) and use a relatively small batch size of 16. 

In [6]:
# fit the model
model.fit(X, y, epochs=150, batch_size=16)

Epoch 1/150
48/48 [==============================] - 1s 949us/step - loss: 4.0663 - accuracy: 0.5234
Epoch 2/150
48/48 [==============================] - 0s 866us/step - loss: 1.9515 - accuracy: 0.6185
Epoch 3/150
48/48 [==============================] - 0s 847us/step - loss: 1.6644 - accuracy: 0.6146
Epoch 4/150
48/48 [==============================] - 0s 877us/step - loss: 1.5068 - accuracy: 0.6146
Epoch 5/150
48/48 [==============================] - 0s 866us/step - loss: 1.2874 - accuracy: 0.6107
Epoch 6/150
48/48 [==============================] - 0s 880us/step - loss: 1.1270 - accuracy: 0.6393
Epoch 7/150
48/48 [==============================] - 0s 883us/step - loss: 1.0279 - accuracy: 0.6367
Epoch 8/150
48/48 [==============================] - 0s 903us/step - loss: 0.9399 - accuracy: 0.6250
Epoch 9/150
48/48 [==============================] - 0s 912us/step - loss: 0.8609 - accuracy: 0.6328
Epoch 10/150
48/48 [==============================] - 0s 881us/step - loss: 0.8120 - accura

48/48 [==============================] - 0s 831us/step - loss: 0.5329 - accuracy: 0.7500
Epoch 82/150
48/48 [==============================] - 0s 817us/step - loss: 0.5392 - accuracy: 0.7253
Epoch 83/150
48/48 [==============================] - 0s 783us/step - loss: 0.5418 - accuracy: 0.7266
Epoch 84/150
48/48 [==============================] - 0s 736us/step - loss: 0.5233 - accuracy: 0.7500
Epoch 85/150
48/48 [==============================] - 0s 831us/step - loss: 0.5423 - accuracy: 0.7526
Epoch 86/150
48/48 [==============================] - 0s 829us/step - loss: 0.5616 - accuracy: 0.7253
Epoch 87/150
48/48 [==============================] - 0s 751us/step - loss: 0.5358 - accuracy: 0.7383
Epoch 88/150
48/48 [==============================] - 0s 820us/step - loss: 0.5395 - accuracy: 0.7292
Epoch 89/150
48/48 [==============================] - 0s 871us/step - loss: 0.5695 - accuracy: 0.7266
Epoch 90/150
48/48 [==============================] - 0s 798us/step - loss: 0.5295 - accuracy: 

## Evaluate the Model

While we have an accuracy rating, it only applies to the training dataset. We do not have an out of sample dataset to work with. 

We can use the `evaluate()` function and pass it the same input/output used to train the model. This will generate a prediction for each input and output pair and collect scores, including the average loss and any metrics you have configured, such as accuracy.

In [7]:
# evaluate the keras model
_, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

24/24 [==============================] - 0s 794us/step - loss: 0.5848 - accuracy: 0.7266
Accuracy: 72.66


Neural networks are a stochastic algorithm, meaning that the same algorithm ont he same data can train a different model with different skill each time the code is run. This is a feature, not a bug. The variance in the performance of the model means that to get a reasomable approximation of how well your model is performing, you may need to fit it many times and calculate the average of the accuracy scores. 

In [8]:
def multiple_model_example(X=X, y=y, num=5):
    accuracies = []
    for i in range(num):
        model = Sequential()
        model.add(Dense(12, input_dim=8, activation='relu'))
        model.add(Dense(8, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        # compile the keras model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # fit the keras model on the dataset
        model.fit(X, y, epochs=150, batch_size=16, verbose=0)
        # evaluate the keras model
        _, accuracy = model.evaluate(X, y)
        accuracies.append(accuracy*100)
    print("\n", f"The average accuracy over {num} fits is: {round (sum(accuracies) / len(accuracies), 2)}%")

In [9]:
multiple_model_example()

24/24 [==============================] - 0s 832us/step - loss: 0.5154 - accuracy: 0.7331

 The average accuracy over 5 fits is: 75.73%


## Make Predictions

We can adapt the above example and use it to generate predictions on the training dataset, pretending it is a new dataset we have not seen before. Making predictions is as easy as calling the `predict()` function on the model. We are using a sigmoid activation function on the output layer, so the predictions will be a probability in the range between 0 and 1. We can easily convert them into a crisp binary prediction for this classification task by rounding them.

In [10]:
# make probability predictions with the model
predictions = model.predict(X)

# round predictions
rounded = [round(x[0]) for x in predictions]

24/24 [==============================] - 0s 602us/step


In [11]:
print("Showing the first 5 predictions:")
print(rounded[:5])
print(f"There are {len(rounded)} predictions made for this set.")

Showing the first 5 predictions:
[1, 0, 1, 0, 1]
There are 768 predictions made for this set.


We can compare the predictions to the actual values in the target variable:

In [12]:
for i in range(5):
    print("%s => %d (expected %d)" % (X[i].tolist(), rounded[i], y[i]))

[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0] => 1 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0] => 1 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0] => 1 (expected 1)


## Summary

In this example, we loaded the data from the Pima Indians Diabetes Onset dataset. We defined a neural network model in Keras. We compiled the model using the efficient numerical backend. We trained a model on the data. Then, we evaluated our model. 